# TOML Configuration

Save and load task configurations for hardware portability.

In [ ]:
# Check for nidaqmx
try:
    import nidaqmx
except ImportError:
    raise RuntimeError("nidaqmx not installed. Install with: pip install nidaqmx")

In [ ]:
import numpy as np
from nidaqwrapper import AITask, AOTask, DITask, DOTask
from nidaqwrapper.utils import list_devices

# Show connected hardware
devices = list_devices()
print("Connected devices:")
for idx, dev in enumerate(devices):
    print(f"  {idx}: {dev['name']} ({dev['product_type']})")

## Save AITask Config

In [ ]:
# Create AITask with channels
task = AITask("config_test", sample_rate=25600)
task.add_channel("accel_x", device_ind=0, channel_ind=0, 
                 sensitivity=100.0, sensitivity_units="mV/g", units="g")
task.add_channel("accel_y", device_ind=0, channel_ind=1,
                 sensitivity=100.0, sensitivity_units="mV/g", units="g")

# Save to TOML
task.save_config("ai_config.toml")
task.clear_task()

# Show file content
print(open("ai_config.toml").read())

## Load Config

In [ ]:
# Recreate task from TOML
loaded = AITask.from_config("ai_config.toml")
print(f"Task name: {loaded.task_name}")
print(f"Sample rate: {loaded.sample_rate} Hz")
print(f"Channels: {loaded.channel_list}")
loaded.clear_task()

## Device Alias Mapping

When moving configs to different hardware, use `device_map` to remap device aliases.

In [ ]:
# Example: remap Dev1 -> Dev2 when moving to different chassis
# Uncomment if you have Dev2:
# remapped = AITask.from_config("ai_config.toml")
# print(f"Channels: {remapped.channel_list}")
# remapped.clear_task()

print("Device mapping allows portable configs across different hardware enumeration.")

## AOTask Config

In [ ]:
# Create and save AOTask
ao = AOTask("sig_gen", sample_rate=10000)
ao.add_channel("ao_0", device_ind=0, channel_ind=0, min_val=-5.0, max_val=5.0)
ao.save_config("ao_config.toml")
ao.clear_task()

print(open("ao_config.toml").read())

## Digital Config

In [ ]:
# Digital input uses line specifications, not device_ind/channel_ind
di = DITask("switches", sample_rate=1000)
di.add_channel("sw1", lines="Dev1/port0/line0:3")
di.save_config("di_config.toml")
di.clear_task()

print(open("di_config.toml").read())

## Summary

TOML configs make task definitions:
- Human-readable and editable
- Portable across hardware (via device aliases)
- Version-controllable

Use `save_config()` to persist, `from_config()` to restore.